# Update data

This notebook downlads recent GitHub activity for a number of organizations.

It will extract all issues, PRs, and comments that were updated within a
window of interest. It will then save them to disk as CSV files.

In [3]:
import requests
import pandas as pd
import numpy as np
import os
from datetime import timedelta

In [59]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 365
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))
update_data = True

github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [60]:
if update_data is True:
    # Load in previous data if we have it
    path_prs = './data/prs.csv'
    path_issues = './data/issues.csv'
    path_comments = './data/comments.csv'
    prs_old = pd.read_csv(path_prs, index_col=0)
    issues_old = pd.read_csv(path_issues, index_col=0)
    comments_old = pd.read_csv(path_comments, index_col=0)

    # The latest updated time, we'll update `start_date` so we don't re-download unnecessarily
    latest_date = pd.to_datetime(comments_old['updatedAt'].max())
    start_date = fmt.format(latest_date)

In [61]:
# Calculate number of days to include in plots
n_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

In [62]:
comments_query = """\
        comments(last: 50) {
          edges {
            node {
              authorAssociation
              createdAt
              updatedAt
              url
              author {
                login
              }
            }
          }
        }
"""

base_elements = """\
        state
        id
        title
        url
        createdAt
        updatedAt
        closedAt
        number
        authorAssociation
        author {
          login
        }
"""

gql_template = """\
{{
  search({query}) {{
    issueCount
    pageInfo {{
        endCursor
        hasNextPage
    }}
    nodes {{
      ... on PullRequest {{
        {base_elements}
        mergedBy {{
          login
        }}
        {comments}
      }}
      ... on Issue {{
        {base_elements}
        {comments}
      }}
    }}
  }}
}}
"""

In [63]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = gql_template

    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template.format(
                query=', '.join(search_query),
                comments=comments_query,
                base_elements=base_elements
            )
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['author'] = self.data['author'].map(lambda a: a['login'] if a is not None else a)
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [64]:
def extract_comments(comments):
    list_of_comments = [ii['edges'] for ii in comments]
    comments = [jj['node'] for ii in list_of_comments for jj in ii]
    comments = pd.DataFrame(comments)
    comments['author'] = comments['author'].map(lambda a: a['login'] if a is not None else a)
    
    # Parse some data about the comments
    url_parts = [ii.split('/') for ii in comments['url'].values]
    url_parts = np.array([(ii[3], ii[4], ii[6]) for ii in url_parts])
    orgs, repos, url_parts = url_parts.T

    issue_id = [ii.split('#')[0] for ii in url_parts]
    comment_id = [ii.split('-')[-1] for ii in url_parts]

    # Assign new variables
    comments['org'] = orgs
    comments['repo'] = repos
    comments['issue_id'] = issue_id
    comments['id'] = comment_id
    return comments

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

## Update issues

In [65]:
responses = []
for org in github_orgs:
    query_issues = f"is:issue user:{org} updated:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_issues)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)

issues = pd.concat([ii.data for ii in responses])
issues_comments = issues.pop("comments")
issues_comments = extract_comments(issues_comments)

Found 2273 items, which will take 46 pages
Found 3163 items, which will take 64 pages
Found 2400 items, which will take 48 pages
Found 565 items, which will take 12 pages
Found 748 items, which will take 15 pages
Found 42 items, which will take 1 pages
Found 1714 items, which will take 35 pages


## Update PRs

In [66]:
responses = []
for org in github_orgs:
    query_prs = f"is:pr user:{org} created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
prs = pd.concat([ii.data for ii in responses])
prs_comments = prs.pop('comments')
prs_comments = extract_comments(prs_comments)

Found 1665 items, which will take 34 pages
Found 1407 items, which will take 29 pages
Found 1243 items, which will take 25 pages
Found 316 items, which will take 7 pages
Found 311 items, which will take 7 pages
Found 31 items, which will take 1 pages
Found 1774 items, which will take 36 pages


In [67]:
# Add a PR-specific field for closed PRs
prs['mergedBy'] = prs['mergedBy'].map(lambda a: a['login'] if a is not None else None)

## Combine comments

In [68]:
comments = pd.concat([prs_comments, issues_comments])

# Only keep the comments within our window of interest
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')

## Update the data

In [69]:
if update_data is True:
    comments_new = pd.concat([comments_old, comments]).drop_duplicates(subset=['id'], keep='last').sort_values('createdAt', ascending=False)
    issues_new = pd.concat([issues_old, issues]).drop_duplicates(subset=['id'], keep='last').sort_values('createdAt', ascending=False)
    prs_new = pd.concat([prs_old, prs]).drop_duplicates(subset=['id'], keep='last').sort_values('createdAt', ascending=False)
else:
    comments_new = comments
    issues_new = issues
    prs_new = prs

## Save the data

In [70]:
prs_new.to_csv('./data/prs.csv')
issues_new.to_csv('./data/issues.csv')
comments_new.to_csv('./data/comments.csv')

## View the data

In [71]:
issues_new.head(10)

,author,authorAssociation,closedAt,createdAt,id,number,state,title,updatedAt,url,org,repo
0,stormerider,NONE,None,2019-07-22T16:45:10Z,MDU6SXNzdWU0NzExODUxNzU=,130,OPEN,TLJH and LDAP Authentication,2019-07-22T16:45:10Z,https://github.com/jupyterhub/ldapauthenticato...,jupyterhub,ldapauthenticator
1,fsksf,NONE,None,2019-07-22T09:00:11Z,MDU6SXNzdWU0NzA5NzU5Mzk=,325,OPEN,Unstable startup time,2019-07-22T09:17:55Z,https://github.com/jupyterhub/dockerspawner/is...,jupyterhub,dockerspawner
2,JulianTBZ,NONE,None,2019-07-22T07:32:52Z,MDU6SXNzdWU0NzA5Mzg0OTM=,2655,OPEN,No login windows shown,2019-07-22T13:14:41Z,https://github.com/jupyterhub/jupyterhub/issue...,jupyterhub,jupyterhub
3,lshailendra,NONE,None,2019-07-21T20:13:07Z,MDU6SXNzdWU0NzA4MTk5MTE=,2654,OPEN,"login page shows ""this page cannot be displayed""",2019-07-22T16:48:48Z,https://github.com/jupyterhub/jupyterhub/issue...,jupyterhub,jupyterhub
4,davidedelvento,NONE,None,2019-07-19T21:34:25Z,MDU6SXNzdWU0NzA1NjY3NjQ=,2653,OPEN,setting ssl_cert to fullchain results in expir...,2019-07-22T17:02:36Z,https://github.com/jupyterhub/jupyterhub/issue...,jupyterhub,jupyterhub
5,minrk,MEMBER,None,2019-07-19T06:46:30Z,MDU6SXNzdWU0NzAxNjU2ODc=,189,OPEN,Onboarding @GeorgianaElena,2019-07-22T13:03:57Z,https://github.com/jupyterhub/team-compass/iss...,jupyterhub,team-compass
6,SandeepBhutani,NONE,None,2019-07-18T17:44:37Z,MDU6SXNzdWU0Njk5MDUzMzI=,2652,OPEN,Jupyterhub user server goes down automatically,2019-07-18T17:44:37Z,https://github.com/jupyterhub/jupyterhub/issue...,jupyterhub,jupyterhub
7,kipstakkr,NONE,None,2019-07-18T13:44:14Z,MDU6SXNzdWU0Njk3Nzg0NTU=,2651,OPEN,Create groups and add users to the group using...,2019-07-18T13:47:34Z,https://github.com/jupyterhub/jupyterhub/issue...,jupyterhub,jupyterhub
8,JulianTBZ,NONE,None,2019-07-18T08:31:34Z,MDU6SXNzdWU0Njk2Mzc5MTU=,129,OPEN,Jupyterhub Active Directory Authentication,2019-07-19T10:00:36Z,https://github.com/jupyterhub/ldapauthenticato...,jupyterhub,ldapauthenticator
9,GrahamDumpleton,CONTRIBUTOR,None,2019-07-17T00:07:59Z,MDU6SXNzdWU0Njg5MjUyNjI=,336,OPEN,Add support for storage_selector through confi...,2019-07-17T00:07:59Z,https://github.com/jupyterhub/kubespawner/issu...,jupyterhub,kubespawner


In [72]:
prs_new.head(10)

,author,authorAssociation,closedAt,createdAt,id,mergedBy,number,state,title,updatedAt,url,org,repo
0,betatim,MEMBER,2019-07-19T10:20:26Z,2019-07-19T10:20:18Z,MDExOlB1bGxSZXF1ZXN0Mjk5Mjc0NTUx,betatim,1090,MERGED,Increase traffic to OVH again,2019-07-19T10:20:28Z,https://github.com/jupyterhub/mybinder.org-dep...,jupyterhub,mybinder.org-deploy
1,henchbot,CONTRIBUTOR,2019-07-19T09:05:15Z,2019-07-19T09:01:18Z,MDExOlB1bGxSZXF1ZXN0Mjk5MjQ2Mzky,betatim,1089,MERGED,repo2docker: 74af2ad8...6c5c09b6,2019-07-19T09:05:15Z,https://github.com/jupyterhub/mybinder.org-dep...,jupyterhub,mybinder.org-deploy
2,minrk,MEMBER,2019-07-19T08:34:16Z,2019-07-19T06:47:07Z,MDExOlB1bGxSZXF1ZXN0Mjk5MjA2NTY1,betatim,190,MERGED,add georgiana to jupyterhub team,2019-07-19T14:44:31Z,https://github.com/jupyterhub/team-compass/pul...,jupyterhub,team-compass
3,choldgraf,MEMBER,2019-07-19T03:09:16Z,2019-07-19T03:08:32Z,MDExOlB1bGxSZXF1ZXN0Mjk5MTY3MzIw,choldgraf,1088,MERGED,Bump OVH cluster down to 0 because it's down,2019-07-19T04:48:28Z,https://github.com/jupyterhub/mybinder.org-dep...,jupyterhub,mybinder.org-deploy
4,henchbot,CONTRIBUTOR,2019-07-18T20:13:47Z,2019-07-18T20:00:43Z,MDExOlB1bGxSZXF1ZXN0Mjk5MDY5ODA5,betatim,1087,MERGED,repo2docker: 88eaa230...74af2ad8,2019-07-18T20:13:47Z,https://github.com/jupyterhub/mybinder.org-dep...,jupyterhub,mybinder.org-deploy
5,minrk,MEMBER,2019-07-18T17:48:46Z,2019-07-18T16:26:18Z,MDExOlB1bGxSZXF1ZXN0Mjk4OTk0ODE3,choldgraf,188,MERGED,update meeting interval in README,2019-07-18T17:48:46Z,https://github.com/jupyterhub/team-compass/pul...,jupyterhub,team-compass
6,betatim,MEMBER,None,2019-07-18T05:49:16Z,MDExOlB1bGxSZXF1ZXN0Mjk4NzYzOTUy,None,6,OPEN,[WIP] Add Binder comic panels,2019-07-18T05:49:16Z,https://github.com/jupyterhub/design/pull/6,jupyterhub,design
7,Carreau,MEMBER,None,2019-07-17T23:57:41Z,MDExOlB1bGxSZXF1ZXN0Mjk4NzEwNjk2,None,2650,OPEN,[Discussion] Add options to validate HTML befo...,2019-07-17T23:57:41Z,https://github.com/jupyterhub/jupyterhub/pull/...,jupyterhub,jupyterhub
8,rochaporto,FIRST_TIME_CONTRIBUTOR,None,2019-07-17T20:30:49Z,MDExOlB1bGxSZXF1ZXN0Mjk4NjU3NjUx,None,276,OPEN,Add group support to generic oauthenticator,2019-07-17T20:30:49Z,https://github.com/jupyterhub/oauthenticator/p...,jupyterhub,oauthenticator
9,chicocvenancio,FIRST_TIME_CONTRIBUTOR,None,2019-07-17T13:02:28Z,MDExOlB1bGxSZXF1ZXN0Mjk4NDU5MjIy,None,901,OPEN,Bump JupyterHub chart,2019-07-17T18:32:37Z,https://github.com/jupyterhub/binderhub/pull/901,jupyterhub,binderhub


In [73]:
comments_new.head(10)

,author,authorAssociation,createdAt,updatedAt,url,org,repo,issue_id,id
0,choldgraf,MEMBER,2019-07-19T14:44:30Z,2019-07-19T14:44:30Z,https://github.com/jupyterhub/team-compass/pul...,jupyterhub,team-compass,190,513256772
1,betatim,MEMBER,2019-07-17T18:32:05Z,2019-07-17T18:32:37Z,https://github.com/jupyterhub/binderhub/pull/9...,jupyterhub,binderhub,901,512515252
2,rkdarst,CONTRIBUTOR,2019-07-19T09:42:30Z,2019-07-19T09:42:30Z,https://github.com/jupyterhub/batchspawner/pul...,jupyterhub,batchspawner,151,513166239
3,dalg24,CONTRIBUTOR,2019-07-17T07:39:25Z,2019-07-17T07:39:25Z,https://github.com/jupyterhub/jupyterhub-deplo...,jupyterhub,jupyterhub-deploy-docker,88,512140790
4,nicorikken,CONTRIBUTOR,2019-07-17T07:38:41Z,2019-07-17T07:38:41Z,https://github.com/jupyterhub/jupyterhub/pull/...,jupyterhub,jupyterhub,2648,512140572
5,nicorikken,CONTRIBUTOR,2019-07-17T10:53:49Z,2019-07-17T10:53:49Z,https://github.com/jupyterhub/jupyterhub/pull/...,jupyterhub,jupyterhub,2648,512204987
7,GrahamDumpleton,CONTRIBUTOR,2019-07-20T09:26:16Z,2019-07-20T09:26:16Z,https://github.com/jupyterhub/kubespawner/pull...,jupyterhub,kubespawner,338,513452280
8,consideRatio,MEMBER,2019-07-20T09:46:49Z,2019-07-20T09:47:24Z,https://github.com/jupyterhub/kubespawner/pull...,jupyterhub,kubespawner,338,513453508
9,consideRatio,MEMBER,2019-07-20T09:10:22Z,2019-07-20T09:10:22Z,https://github.com/jupyterhub/kubespawner/pull...,jupyterhub,kubespawner,337,513451281
10,GrahamDumpleton,CONTRIBUTOR,2019-07-20T09:25:14Z,2019-07-20T09:25:14Z,https://github.com/jupyterhub/kubespawner/pull...,jupyterhub,kubespawner,337,513452216
